<a href="https://colab.research.google.com/github/Vicnent/cadenas-mastermind/blob/main/Cadenas_Master_Mind.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
from itertools import permutations
import pandas as pd

def generate_solution():
    while True:
        combi = ''.join(random.choices('123456789', k=1) + random.choices('0123456789', k=3))
        if len(set(combi)) == 4:
            return combi

def count_well_and_misplaced(solution, guess):
    well_placed = sum(s == g for s, g in zip(solution, guess))
    remaining_sol = [s for s, g in zip(solution, guess) if s != g]
    remaining_guess = [g for s, g in zip(solution, guess) if s != g]
    misplaced = 0
    for g in remaining_guess:
        if g in remaining_sol:
            misplaced += 1
            remaining_sol.remove(g)
    return well_placed, misplaced

def generate_random_guess(solution):
    while True:
        guess = ''.join(random.choices('123456789', k=1) + random.choices('0123456789', k=3))
        if guess != solution and len(set(guess)) == 4:
            return guess

def phrase_from_counts(guess, wp, mp):
    if wp == 0 and mp == 0:
        return f"Avec {guess}, rien d'intéressant."
    parts = []
    if wp == 0:
        parts.append("")
    elif wp == 1:
        parts.append("1 chiffre bien placé")
    else:
        parts.append(f"{wp} chiffres bien placés")

    if mp == 0:
        parts.append("")
    elif mp == 1:
        parts.append("1 chiffre mal placé")
    else:
        parts.append(f"{mp} chiffres mal placés")

    return f"Avec {guess}, {', '.join(parts)}."

def generate_puzzle(solution, num_clues=4):
    clues = []
    guesses = set()
    attempts = 0
    while len(clues) < num_clues and attempts < 1000:
        attempts += 1
        guess = generate_random_guess(solution)
        if guess in guesses:
            continue
        guesses.add(guess)
        wp, mp = count_well_and_misplaced(solution, guess)

        # Contraintes imposées
        # if wp > 0: # si on veut uniquement des mal placés
            # continue
        if wp == 0 and mp > 2:
            continue
        if mp == 0 and wp > 2:
            continue
        if wp + mp > 3:
            continue

        phrase = phrase_from_counts(guess, wp, mp)
        clues.append((guess, wp, mp, phrase))
    return clues

def find_possible_solutions(clues):
    all_combinations = [''.join(p) for p in permutations('0123456789', 4) if p[0] != '0']
    valid = []
    for combi in all_combinations:
        if all(count_well_and_misplaced(combi, guess) == (wp, mp) for guess, wp, mp, _ in clues):
            valid.append(combi)
    return valid

# Génération du puzzle
while True:
    solution = generate_solution()
    clues = generate_puzzle(solution, num_clues=6)
    candidates = find_possible_solutions(clues)
    if len(candidates) == 1:
        result = [f"✅ Solution : {solution}"]
        for _, _, _, phrase in clues:
            result.append(phrase)
        break

# Affichage
df = pd.DataFrame(result, columns=["Énigme"])
print(df.to_string(index=False, justify='left'))


Énigme                                               
                                    ✅ Solution : 3647
                       Avec 8052, rien d'intéressant.
                   Avec 3280, 1 chiffre bien placé, .
                    Avec 1824, , 1 chiffre mal placé.
Avec 2317, 1 chiffre bien placé, 1 chiffre mal placé.
Avec 5684, 1 chiffre bien placé, 1 chiffre mal placé.
                    Avec 2035, , 1 chiffre mal placé.


In [ ]:
# Mastermind solver for 4-digit codes with clues in the format <guess>;<well_placed>;<misplaced>
# Example inputs: "5432;4;1", "4322;;2", "3211;1;"
# Requires: 4-digit guess, 2 semicolons, at least one number for well_placed or misplaced
# Outputs: Number of remaining solutions after each clue, stops when a unique solution is found or none remain

from itertools import permutations

def generate_all_combinations():
    """Generate all possible 4-digit codes with distinct digits, first digit non-zero."""
    return [''.join(p) for p in permutations('0123456789', 4) if p[0] != '0']

def count_well_and_misplaced(solution, guess):
    """Calculate well-placed (correct position) and misplaced (correct digit, wrong position) counts."""
    well_placed = sum(s == g for s, g in zip(solution, guess))
    remaining_sol = [s for s, g in zip(solution, guess) if s != g]
    remaining_guess = [g for s, g in zip(solution, guess) if s != g]
    misplaced = 0
    for g in remaining_guess:
        if g in remaining_sol:
            misplaced += 1
            remaining_sol.remove(g)
    return well_placed, misplaced

def parse_clue(clue_str):
    """Parse clue in format 'guess;wp;mp' (e.g., '5432;4;1', '4322;;2')."""
    try:
        guess, wp, mp = clue_str.split(';')
        if not (len(guess) == 4 and guess.isdigit() and len(set(guess)) == 4 and guess[0] != '0'):
            raise ValueError("Guess must be a 4-digit number with distinct digits, first digit non-zero.")
        wp = int(wp) if wp.strip() else 0
        mp = int(mp) if mp.strip() else 0
        if wp == 0 and mp == 0:
            raise ValueError("At least one of well-placed or misplaced must be non-zero.")
        if wp < 0 or mp < 0 or wp + mp > 4 or (wp == 4 and mp > 0):
            raise ValueError("Invalid clue: wp + mp <= 4, non-negative, mp = 0 if wp = 4.")
        return guess, wp, mp
    except ValueError as e:
        raise ValueError(f"Invalid clue format: {str(e)}")

def filter_solutions(solutions, guess, wp, mp):
    """Filter solutions that match the clue (wp, mp) for the given guess."""
    return [sol for sol in solutions if count_well_and_misplaced(sol, guess) == (wp, mp)]

def mastermind_solver():
    """Interactively solve a 4-digit Mastermind puzzle using user-provided clues."""
    solutions = generate_all_combinations()  # Start with all 4536 possible 4-digit codes
    print(f"Starting with {len(solutions)} possible solutions.")

    while len(solutions) > 1:
        # Prompt user for clue in format <guess>;<well_placed>;<misplaced>
        clue = input("Enter clue as 'guess;well_placed;misplaced' (e.g., '5432;1;1', '4322;;2', '3211;1;'): ").strip()
        try:
            guess, wp, mp = parse_clue(clue)
        except ValueError as e:
            print(f"Error: {e}")
            continue

        # Filter solutions based on the clue
        solutions = filter_solutions(solutions, guess, wp, mp)
        num_solutions = len(solutions)

        # Report results
        if num_solutions > 1:
            print(f"There are still {num_solutions} solutions. I need more!")
        elif num_solutions == 1:
            print(f"✅ Found the solution: {solutions[0]}")
            break
        else:
            print("No solutions match the clues. Please check your input for consistency.")
            return

    # Final status
    if len(solutions) == 0:
        print("No solutions remain. The clues may be inconsistent.")
    elif len(solutions) == 1:
        print(f"✅ Confirmed solution: {solutions[0]}")
    else:
        print(f"Could not narrow down to a single solution. {len(solutions)} solutions remain.")

# Run the solver in Colab
if __name__ == "__main__":
    mastermind_solver()



# Avec 7894, 1 chiffre bien placé, .
# Avec 1824, rien d'intéressant.
# Avec 3840, 1 chiffre bien placé, 1 chiffre mal placé.
# Avec 4697, 1 chiffre bien placé, 1 chiffre mal placé.
# Avec 3926, 2 chiffres bien placés, 1 chiffre mal placé.
# Avec 2053, 1 chiffre bien placé, 1 chiffre mal placé.

# Starting with 4536 possible solutions.
# Enter clue as 'guess;well_placed;misplaced' (e.g., '5432;4;1', '4322;;2', '3211;1;'): 7894;1;
# There are still 420 solutions. I need more!
# Enter clue as 'guess;well_placed;misplaced' (e.g., '5432;4;1', '4322;;2', '3211;1;'): 1824;0;
# Error: Invalid clue format: At least one of well-placed or misplaced must be non-zero.
# Enter clue as 'guess;well_placed;misplaced' (e.g., '5432;4;1', '4322;;2', '3211;1;'): 3840;1;1
# There are still 64 solutions. I need more!
# Enter clue as 'guess;well_placed;misplaced' (e.g., '5432;4;1', '4322;;2', '3211;1;'): 4697;1;1
# There are still 6 solutions. I need more!
# Enter clue as 'guess;well_placed;misplaced' (e.g., '5432;4;1', '4322;;2', '3211;1;'): 3926;2;1
# There are still 2 solutions. I need more!
# Enter clue as 'guess;well_placed;misplaced' (e.g., '5432;4;1', '4322;;2', '3211;1;'): 2053;1;1
# ✅ Found the solution: 3096
# ✅ Confirmed solution: 3096


Starting with 4536 possible solutions.
